In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import itertools
import yt

In [ ]:
class_007 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix007.npy")

In [ ]:
class_054 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix054.npy")

In [ ]:
class_076 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix076.npy")

In [ ]:
class_116 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix116.npy")

In [ ]:
class_13 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix13.npy")

In [ ]:
class_21 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix21.npy")

In [ ]:
class_35 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix35.npy")

In [ ]:
class_41 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix41.npy")

In [ ]:
class_67 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix67.npy")

In [ ]:
class_85 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix85.npy")

In [ ]:
class_101 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix101.npy")

In [ ]:
rho007 = np.load("/vol/calvin/data/adave/Internship/SmoothDensityField007.npy")

In [ ]:
rho054 = np.load("/vol/calvin/data/adave/Internship/SmoothDensityField054.npy")

In [ ]:
rho076 = np.load("/vol/calvin/data/adave/Internship/SmoothDensityField076.npy")

In [ ]:
rho116 = np.load("/vol/calvin/data/adave/Internship/SmoothDensityField116.npy")

In [ ]:
rho13 = np.load("/vol/calvin/data/adave/Internship/SmoothDensityField13.npy")

In [ ]:
rho21 = np.load("/vol/calvin/data/adave/Internship/SmoothDensityField21.npy")

In [ ]:
rho35 = np.load("/vol/calvin/data/adave/Internship/SmoothDensityField35.npy")

In [ ]:
v_field = np.load("/vol/calvin/data/adave/Internship/Velocity_field.npy")

In [ ]:
pos = np.load("/vol/calvin/data/adave/Internship/Particle_position.npy")

In [ ]:
num_labels = [0, 1, 2, 3]

def calculate_volume_fraction(classification_matrix, label):
    """
    Calculate the volume fraction of a --specific label-- in a 3D classification matrix.

    Parameters:
    - classification_matrix (numpy.ndarray): 3D array containing voxel labels.
    - label (int): The label for which the volume fraction is calculated.

    Returns:
    float: The volume fraction of the specified label in the matrix.
    """
    return np.sum(classification_matrix.flatten() == label) / np.prod(classification_matrix.shape)

In [ ]:
def calculate_volume_fractions(classification_matrix, num_labels):
    """
    Calculate the volume fractions for --multiple labels-- in a 3D classification matrix.

    Parameters:
    - classification_matrix (numpy.ndarray): 3D array containing voxel labels.
    - num_labels (numpy.ndarray or list): Array or list of labels for which volume fractions are calculated.

    Returns:
    numpy.ndarray: Array of volume fractions corresponding to each label.
    """
    volume_fractions = np.zeros_like(num_labels, dtype=float)

    for i, label in enumerate(num_labels):
        volume_fractions[i] = calculate_volume_fraction(classification_matrix, label)
    
    return volume_fractions

In [ ]:
def multi_scale_analysis(classification_matrix, num_labels, scales):
    """
    Perform multi-scale analysis of volume fractions for specified labels in a 3D classification matrix.

    Parameters:
    - classification_matrix (numpy.ndarray): 3D array containing voxel labels.
    - num_labels (numpy.ndarray or list): Array or list of labels for which volume fractions are calculated.
    - scales (list): List of scale factors for subsampling the classification matrix.

    Returns:
    numpy.ndarray: 2D array where each row corresponds to volume fractions at different scales for each label.
    """
    results = []
    for scale in tqdm(scales, desc='Processing scales'):
        scaled_matrix = classification_matrix[::scale, ::scale, ::scale]
        volume_fractions = calculate_volume_fractions(scaled_matrix, num_labels)
        results.append(volume_fractions)
        
    return np.array(results) 

In [ ]:
labels = ['Void', 'Sheets', 'Filaments', 'Clusters']

sm_scales = [0.078, 0.54, 0.76, 1.167, 1.3, 2.1, 3.5, 4.1, 6.7, 8.5, 10.1]
classifications = [class_007, class_054, class_076, class_116, class_13, class_21, class_35, class_41, class_67, class_85, class_101]


vol_fracs = []

for class_i in tqdm(classifications):
    vol_frac = calculate_volume_fractions(class_i, num_labels)
    vol_fracs.append(vol_frac)

In [ ]:
vol_fracs = np.array(vol_fracs)
vol_fracs.shape

In [ ]:
plt.figure(figsize = (4,4), dpi = 200)
for i in range(len(vol_fracs.T)):
    plt.semilogx(sm_scales, vol_fracs[:, i], label = labels[i])

plt.axhline(0.43, ls = '--', lw = 1, color = 'black', alpha = 0.4)
plt.axhline(0.072, ls = '--', lw = 1, color = 'black', alpha = 0.4)
plt.axvline(0.54, ls = '--', lw = 1, color = 'black', alpha = 0.8)
plt.text(0.65, 0.2, '$R_s = 0.54~h^{-1}~Mpc$', fontsize = 8)
plt.xlabel('$R_s~[h^{-1}~ Mpc] $')
plt.ylabel('Volume fraction')
plt.tick_params(axis='both',  which='both',  left=True, right=True, top = True, bottom = True, direction = 'in', labelsize = 7)
# plt.xticks(fontsize = 10)
plt.legend(bbox_to_anchor = (1, 0.6), fontsize = 9, fancybox = True, handlelength = 1)
plt.show()

In [ ]:
def slice_density_field(density_field, slice_index):
    """
    Extracts a 2D slice from a 3D density field and applies logarithmic transformation.

    Parameters:
    - density_field (numpy.ndarray): 3D array representing the density field.
    - slice_index (int): Index of the slice to be extracted along the z-axis.

    Returns:
    - numpy.ndarray: 2D array representing the logarithmic transformation of the specified slice.

    Notes:
    - The logarithmic transformation is applied using the formula np.log10(density_field + 1 + eps),
      where eps is a small constant (1e-15) to prevent taking the logarithm of zero.

    Example:
    >>> density_field = np.random.rand(5, 5, 5)
    >>> slice_index = 2
    >>> result = slice_density_field(density_field, slice_index)
    """
    eps = 1e-15  # so that log doesn't get a value 0
    N = 0

    delplot1 = np.log10(density_field + 1 + eps)
    d_field = delplot1[:, :, slice_index]
    
    return d_field

In [ ]:
def get_pos(structure, projection, slice_index, classification, grid_size=512, box_size=100):
    
    """
    Extracts positions of points in a 2D slice based on the specified structure, projection, and classification.

    Parameters:
    - structure (str): A string representing the structure type ('v', 's', 'f', 'n').
    - projection (str): A string representing the projection type ('xy', 'yx', 'yz', 'zy', 'zx', 'xz').
    - slice_index (int): Index of the slice to be extracted.
    - classification (numpy.ndarray): 3D array representing the classification of points.
    - grid_size (int): Size of the grid in the 3D array (default is 512).
    - box_size (int): Size of the box in physical units (default is 100).

    Returns:
    - numpy.ndarray: 2D array representing the positions of points in the specified slice.

    Notes:
    - Points are classified into different structures based on the structure parameter.
    - The projection parameter determines the orientation of the slice (e.g., 'xy', 'yz', 'zx').
    - The classification parameter is a 3D array where points are classified into different structures.
    - The positions are returned in physical units, scaled based on the grid size and box size.
    - The function prints 'ValueError' if invalid structure or projection is provided.

    Example:
    >>> structure = 'v'
    >>> projection = 'xy'
    >>> slice_index = 2
    >>> classification = np.random.randint(0, 4, size=(512, 512, 512))
    >>> result = get_pos(structure, projection, slice_index, classification)
    """
    
    if str(structure)[0].lower() == 'v':

        if (str(projection) == 'xy' or 'yx'):
            mask_r, mask_c = np.where(classification[:, :, slice_index] == 0)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_c, mask_r))
            return position
        
        elif (str(projection) == 'yz' or 'zy'):
            mask_r, mask_c = np.where(classification[slice_index, :, :] == 0)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_r, mask_c))
            return position
        
        elif (str(projection) == 'zx' or 'xz'):
            mask_r, mask_c = np.where(classification[:, slice_index, :] == 0)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_r, mask_c))
            return position
        
        else: 
            print('ValueError')
            
            
    elif str(structure)[0].lower() == 's':
        
        if (str(projection) == 'xy' or 'yx'):
            mask_r, mask_c = np.where(classification[:, :, slice_index] == 1)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_c, mask_r))
            return position
        
        elif (str(projection) == 'yz' or 'zy'):
            mask_r, mask_c = np.where(classification[slice_index, :, :] == 1)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_r, mask_c))
            return position
        
        elif (str(projection) == 'zx' or 'xz'):
            mask_r, mask_c = np.where(classification[:, slice_index, :] == 1)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_r, mask_c))
            return position
        
        else: 
            print('ValueError')
            
            
    elif str(structure)[0].lower() == 'f':
        
        if (str(projection) == 'xy' or 'yx'):
            mask_r, mask_c = np.where(classification[:, :, slice_index] == 2)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_c, mask_r))
            return position
        
        elif (str(projection) == 'yz' or 'zy'):
            mask_r, mask_c = np.where(classification[slice_index, :, :] == 2)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_r, mask_c))
            return position
        
        elif (str(projection) == 'zx' or 'xz'):
            mask_r, mask_c = np.where(classification[:, slice_index, :] == 2)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_r, mask_c))
            return position
        
        else: 
            print('ValueError')
            
            
    elif str(structure)[0].lower() == 'n':
        
        if (str(projection) == 'xy' or 'yx'):
            mask_r, mask_c = np.where(classification[:, :, slice_index] == 3)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_c, mask_r))
            return position
        
        elif (str(projection) == 'yz' or 'zy'):
            mask_r, mask_c = np.where(classification[slice_index, :, :] == 3)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_r, mask_c))
            return position
        
        elif (str(projection) == 'zx' or 'xz'):
            mask_r, mask_c = np.where(classification[:, slice_index, :] == 3)
            mask_r = mask_r * box_size / grid_size
            mask_c = mask_c * box_size / grid_size
            position = np.column_stack((mask_r, mask_c))
            return position
        
        else: 
            print('ValueError')
        
    else:
        print('ValueError')
        

In [ ]:
def get_class_env_pos(classification, slice_index):
    
    """
    Extracts positions of points for all environment types ('v', 's', 'f', 'n') in a 2D slice.

    Parameters:
    - classification (numpy.ndarray): 3D array representing the classification of points.
    - slice_index (int): Index of the slice to be extracted.

    Returns:
    - list: List containing 2D arrays representing positions of points for each environment type.

    Notes:
    - Uses the get_pos function to extract positions for each environment type.
    - The classification parameter is a 3D array where points are classified into different structures.
    - The slice_index parameter determines the index of the slice to be extracted.
    - Positions are returned in physical units, scaled based on the default grid size (512) and box size (100).

    Example:
    >>> classification = np.random.randint(0, 4, size=(512, 512, 512))
    >>> slice_index = 2
    >>> result = get_class_env_pos(classification, slice_index)
    """
    
    envs = ['v', 's', 'f', 'n']
    
    all_env_pos = []
    
    for env in envs:
        env_pos = get_pos(env, 'xy', slice_index, classification)
        all_env_pos.append(env_pos)
    
    return all_env_pos

### Overlay of classifications on density fields

#### 1. For smoothing scale 0.54 $h^{-1}~Mpc$

In [ ]:
s_index = 256

labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_054 = get_class_env_pos(class_054, s_index)
alphas = [0.1, 0.1, 0.1, 0.7]
sizes = [0.3, 0.3, 0.3, 0.8]

slice_054 = slice_density_field(rho054, s_index)

fig, ax = plt.subplots(2, 2, figsize = (10, 8), dpi = 300)

ax = ax.flatten()

for i in range(len(labels)):
    
    ax[i].imshow(slice_054, cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
    ax[i].scatter(env_pos_054[i][:, 0], env_pos_054[i][:, 1], s= sizes[i], alpha=alphas[i], label=labels[i])
    ax[i].set_title(f'{labels[i]} classification', fontsize = 13)
    ax[i].set_xlabel('$h^{-1}~Mpc$')
    ax[i].set_ylabel('$h^{-1}~Mpc$')
    ax[i].tick_params(axis='both',  which='both',  left=True, right=True, top = True, bottom = True, direction = 'in', labelsize = 7)

plt.suptitle(r'$R_s = 0.54~h^{-1}~Mpc$')
fig.tight_layout()
plt.show()

#### 2. For smoothing scale 0.76 $h^{-1}~Mpc$

In [ ]:
s_index = 256

labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_076 = get_class_env_pos(class_076, s_index)
slice_076 = slice_density_field(rho076, s_index)

fig, ax = plt.subplots(2, 2, figsize = (10, 8), dpi = 300)

ax = ax.flatten()

for i in range(len(labels)):
    ax[i].imshow(slice_076, cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
    ax[i].scatter(env_pos_076[i][:, 0], env_pos_076[i][:, 1], s= 0.7, alpha=0.08, label=labels[i])
    ax[i].set_title(f'{labels[i]} classification', fontsize = 13)
    ax[i].set_xlabel(r'$h^{-1}~Mpc$')
    ax[i].set_ylabel(r'$h^{-1}~Mpc$')
    ax[i].tick_params(axis='both',  which='both',  left=True, right=False, top = False, bottom = True, direction = 'in', labelsize = 7)

plt.suptitle(r'$R_s = 0.76~h^{-1}~Mpc$')
fig.tight_layout()
plt.show()

#### 3. For smoothing scale 1.16 $h^{-1}~Mpc$

In [ ]:
s_index = 256

labels = ['Void', 'Sheet', 'Filament', 'Cluster']

env_pos_116 = get_class_env_pos(class_116, s_index)
alphas = [0.1, 0.1, 0.1, 0.1]
sizes = [0.2, 0.2, 0.2, 0.5]
slice_116 = slice_density_field(rho116, s_index)

fig, ax = plt.subplots(2, 2, figsize = (10, 8), dpi = 300)

ax = ax.flatten()

for i in range(len(labels)):
    ax[i].imshow(slice_116, cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
    ax[i].scatter(env_pos_116[i][:, 0], env_pos_116[i][:, 1], s= sizes[i], alpha=alphas[i], label=labels[i])
    ax[i].set_title(f'{labels[i]} classification', fontsize = 13)
    ax[i].set_xlabel(r'$h^{-1}~Mpc$')
    ax[i].set_ylabel(r'$h^{-1}~Mpc$')
    ax[i].tick_params(axis='both',  which='both',  left=True, right=False, top = False, bottom = True, direction = 'in', labelsize = 7)

plt.suptitle(r'$R_s = 1.16~h^{-1}~Mpc$')
fig.tight_layout()
plt.show()

#### 5. For smoothing scale 1.3 $h^{-1}~Mpc$

In [ ]:
s_index = 256

labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_13 = get_class_env_pos(class_13, s_index)
alphas = [0.1, 0.1, 0.1, 0.2]
sizes = [0.2, 0.2, 0.2, 0.5]

slice_13 = slice_density_field(rho13, s_index)

fig, ax = plt.subplots(2, 2, figsize = (10, 8), dpi = 300)

ax = ax.flatten()

for i in range(len(labels)):
    ax[i].imshow(slice_13, cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
    ax[i].scatter(env_pos_13[i][:, 0], env_pos_13[i][:, 1], s= sizes[i], alpha=alphas[i], label=labels[i])
    ax[i].set_title(f'{labels[i]} classification', fontsize = 13)
    ax[i].set_xlabel(r'$h^{-1}~Mpc$')
    ax[i].set_ylabel(r'$h^{-1}~Mpc$')
    ax[i].tick_params(axis='both',  which='both',  left=True, right=False, top = False, bottom = True, direction = 'in', labelsize = 7)

plt.suptitle(r'$R_s = 1.3~h^{-1}~Mpc$')
fig.tight_layout()
plt.show()

In [ ]:
labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_all = [env_pos_054, env_pos_076, env_pos_116, env_pos_13]
smth_scales = [0.54, 0.76, 1.16, 1.3]
d_scales = [slice_054, slice_076, slice_116, slice_13]

alphas = [0.5, 0.5, 0.5, 0.5]
sizes = [0.015, 0.015, 0.015, 0.015]

fig, ax = plt.subplots(2, 2, figsize=(13, 13), dpi=400, sharey = True)
fig.subplots_adjust(wspace = 0.1, hspace = 0.3)

ax = ax.flatten()

for idx, j in enumerate(smth_scales):
    for i in range(len(labels)):
        ax[idx].imshow(d_scales[idx], cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
        ax[idx].scatter(env_pos_all[idx][i][:, 0], env_pos_all[idx][i][:, 1], s=sizes[i], alpha=alphas[i], label=labels[i])
        ax[idx].tick_params(axis='both', which='both', left=True, right=True, top=True, bottom=True, direction='in', labelsize=15)
        ax[idx].set_xlabel(r'$h^{-1}~Mpc$', fontsize = 12)
        ax[idx].set_ylabel(r'$h^{-1}~Mpc$', fontsize = 12)
    ax[idx].set_title(r'$R_s = {:.2f}~h^{{-1}}~Mpc$'.format(j), fontsize = 18)

plt.legend(bbox_to_anchor = (1.05, 0.5), fontsize = 15, markerscale = 70.0, ncol = 1)

# fig.tight_layout()
plt.show()

In [ ]:
labels = ['Void', 'Sheet', 'Filament', 'Cluster']

env_pos_all = [env_pos_054, env_pos_13]
smth_scales = [0, 0.54, 1.3]
d_scales = [slice_054, slice_13]


fig, ax = plt.subplots(1, len(d_scales)+1, figsize=(15, 15), dpi=400)

ax = ax.flatten()

for idx, j in enumerate(smth_scales):
    if idx == 0:
        ax[idx].imshow(slice_054, cmap = 'Greys', extent = [0, 100, 0, 100], origin = 'lower')
        ax[idx].tick_params(axis='both', which='both', left=True, right=True, top=True, bottom=True, direction='in', labelsize=12)
        ax[idx].set_xlabel(r'$h^{-1}~Mpc$', fontsize = 12)
        ax[idx].set_ylabel(r'$h^{-1}~Mpc$', fontsize = 12)
        ax[idx].set_title(r'$R_s = 0.54~h^{{-1}}~Mpc$', fontsize = 15)

    else:
        for i in range(len(labels)):
            ax[idx].imshow(d_scales[idx-1], cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
            ax[idx].scatter(env_pos_all[idx-1][i][:, 0], env_pos_all[idx-1][i][:, 1], s=0.02, alpha=0.3, label=labels[i])
            ax[idx].tick_params(axis='both', which='both', left=True, right=True, top=True, bottom=True, direction='in', labelsize=12)
            ax[idx].set_xlabel(r'$h^{-1}~Mpc$', fontsize = 12)
            ax[idx].set_ylabel(r'$h^{-1}~Mpc$', fontsize = 12)
        ax[idx].set_title(r'$R_s = {:.2f}~h^{{-1}}~Mpc$'.format(j), fontsize = 15)

plt.legend(bbox_to_anchor = (1.05, 0.5), fontsize = 12, markerscale = 70.0, ncol = 1)

fig.tight_layout()
plt.show()

In [ ]:
labels = ['Sheet', 'Filament']

env_pos_054_n = [env_pos_054[1], env_pos_054[2]]
env_pos_13_n = [env_pos_13[1], env_pos_13[2]]


env_pos_all = [env_pos_054_n, env_pos_13_n]
smth_scales = [0.54, 1.3]
d_scales = [slice_054, slice_13]

size = 0.02

alphas = [0.7, 0.7, 0.7, 0.7]
sizes = [size, size, size, size]

fig, ax = plt.subplots(1, len(d_scales), figsize=(12, 12), dpi=400)

ax = ax.flatten()

for idx, j in enumerate(smth_scales):
    for i in range(len(labels)):
        ax[idx].imshow(d_scales[idx], cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
        ax[idx].scatter(env_pos_all[idx][i][:, 0], env_pos_all[idx][i][:, 1], s=sizes[i], alpha=alphas[i], label=labels[i])
        ax[idx].tick_params(axis='both', which='both', left=True, right=True, top=True, bottom=True, direction='in', labelsize=12)
        ax[idx].set_xlabel(r'$h^{-1}~Mpc$')
        ax[idx].set_ylabel(r'$h^{-1}~Mpc$')
    ax[idx].set_title(r'$R_s = {:.2f}~h^{{-1}}~Mpc$'.format(j), fontsize = 15)

plt.legend(bbox_to_anchor = (1.05, 0.5), fontsize = 12, markerscale = 70.0, ncol = 1)

fig.tight_layout()
plt.show()

In [ ]:
def get_env_change(clf1, clf2, diff, init_env):
    """
    Extracts positions where the environment type changes between two classification arrays.

    Parameters:
    - clf1 (numpy.ndarray): First 3D array representing the initial classification of points.
    - clf2 (numpy.ndarray): Second 3D array representing the final classification of points.
    - diff (int): The difference in environment types indicating the change.
    - init_env (int): The initial environment type.

    Returns:
    - numpy.ndarray: 2D array representing positions where the environment type changes.

    Notes:
    - Compares the classification arrays clf1 and clf2 to identify changes.
    - The difference diff is used to detect the change in environment types.
    - The init_env parameter specifies the initial environment type.
    - Positions are returned in physical units, scaled based on the default grid size (512) and box size (100).

    Example:
    >>> clf1 = np.random.randint(0, 4, size=(512, 512, 512))
    >>> clf2 = np.random.randint(0, 4, size=(512, 512, 512))
    >>> diff = 1
    >>> init_env = 2
    >>> result = get_env_change(clf1, clf2, diff, init_env)
    """
    grid_size = 512
    box_size = 100
    slice_index = 256
    
    dif_clf = clf2[:, :, slice_index] - clf1[:, :, slice_index]
    
    bool_clf = np.isclose(dif_clf, diff) & (clf1[:, :, slice_index] == init_env)
    
    col, row = np.where(bool_clf)
    
    col = col * box_size / grid_size
    row = row * box_size / grid_size
    
    result = np.column_stack((row, col))
    
    return result


In [ ]:
# void to sheet
v2s = get_env_change(class_054, class_13, 1, 0)

# sheet to filament
s2f = get_env_change(class_054, class_13, 1, 1)

# filament to node
f2n = get_env_change(class_054, class_13, 1, 2)

# sheet to void
s2v = get_env_change(class_054, class_13, -1, 1)

# filament to sheet
f2s = get_env_change(class_054, class_13, -1, 2)

# node to filament
n2f = get_env_change(class_054, class_13, -1, 3)

In [ ]:
# void to filament
v2f = get_env_change(class_054, class_13, 2, 0)

# void to node
v2n = get_env_change(class_054, class_13, 3, 0)

# sheet to node
s2n = get_env_change(class_054, class_13, 2, 1)

# filament to void
f2v = get_env_change(class_054, class_13, -2, 2)

# node to void
n2v = get_env_change(class_054, class_13, -3, 3)

# node to sheet
n2s = get_env_change(class_054, class_13, -2, 3)

In [ ]:
transform_plots = [v2s, s2f, f2n, s2v, f2s, n2f]
titles = ['Void to Sheet', 'Sheet to Filament', 'Filament to Cluster', \
          'Sheet to Void', 'Filament to Sheet', 'Cluster to Filament']

fig, ax = plt.subplots(2, 3, figsize=(12, 10), dpi=400, sharey = True)
fig.subplots_adjust(wspace = 0.1, hspace = 0)

ax = ax.flatten()

for idx, j in enumerate(transform_plots):
    ax[idx].imshow(slice_054, cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
    ax[idx].scatter(transform_plots[idx][:, 0], transform_plots[idx][:, 1], c= 'r', s = 0.08, alpha = 0.2)
    ax[idx].tick_params(axis='both', which='both', left=True, right=True, top=True, bottom=True, direction='in', labelsize=12)    
    ax[idx].set_title(f'{titles[idx]}', fontsize = 15)

ax[1].set_xlabel(r'$h^{-1}~Mpc$')
ax[0].set_ylabel(r'$h^{-1}~Mpc$')
ax[3].set_ylabel(r'$h^{-1}~Mpc$')
ax[4].set_xlabel(r'$h^{-1}~Mpc$')

# plt.tight_layout()
plt.suptitle('Changes in classification of Halo environments', fontsize = 18)
plt.show()

In [ ]:
transform_plots = [v2f, v2n, s2n, f2v, n2v, n2s]
titles = ['Void to Filament', 'Void to Cluster', 'Sheet to Cluster', \
          'Filament to Void', 'Cluster to Void', 'Cluster to Sheet']

fig, ax = plt.subplots(2, 3, figsize=(12, 10), dpi=400, sharey = True)
fig.subplots_adjust(wspace = 0.1, hspace = 0)

ax = ax.flatten()

for idx, j in enumerate(transform_plots):
    ax[idx].imshow(slice_054, cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
    ax[idx].scatter(transform_plots[idx][:, 0], transform_plots[idx][:, 1], c= 'r', s = 0.08, alpha = 0.2)
    ax[idx].tick_params(axis='both', which='both', left=True, right=True, top=True, bottom=True, direction='in', labelsize=12)    
    ax[idx].set_title(f'{titles[idx]}', fontsize = 15)

ax[1].set_xlabel(r'$h^{-1}~Mpc$')
ax[0].set_ylabel(r'$h^{-1}~Mpc$')
ax[3].set_ylabel(r'$h^{-1}~Mpc$')
ax[4].set_xlabel(r'$h^{-1}~Mpc$')

# plt.tight_layout()
plt.suptitle('Changes in classification of Halo environments', fontsize = 18)
plt.show()

#### $R_s = 0.54 h^{-1} Mpc$ for 5 consecutive slices

In [ ]:
def get_env_pos_and_density(classification, slices, rho):
    """
    Extracts positions and density fields for multiple slices and environment types.

    Parameters:
    - classification (numpy.ndarray): 3D array representing the classification of points.
    - slices (list): List of slice indices to be extracted.
    - rho (numpy.ndarray): 3D array representing the density field.

    Returns:
    - dict: Dictionary containing positions and density fields for each slice and environment type.

    Notes:
    - Uses the get_pos function to extract positions for each environment type.
    - Uses the slice_density_field function to extract density fields for each slice.
    - The classification parameter is a 3D array where points are classified into different structures.
    - The slices parameter is a list of slice indices to be extracted.
    - The rho parameter is a 3D array representing the density field.
    - Positions are returned in physical units, scaled based on the default grid size (512) and box size (100).
    - The result is a dictionary with keys representing slice indices and environment types.

    Example:
    >>> classification = np.random.randint(0, 4, size=(512, 512, 512))
    >>> slices = [1, 2, 3]
    >>> rho = np.random.rand(512, 512, 512)
    >>> result = get_env_pos_and_density(classification, slices, rho)
    """
    envs = ['v', 's', 'f', 'n']
    
    all_slice_pos = {}
    
    all_slice_pos['density'] = []
    
    for slice_index in slices:
        
        all_slice_pos[str(slice_index)] = []
        
        for env in envs:
            env_pos = get_pos(env, 'xy', slice_index, classification)
            all_slice_pos[str(slice_index)].append(env_pos)
        
        all_slice_pos['density'].append(slice_density_field(rho, slice_index))
    
    return all_slice_pos

In [ ]:
def plot_slices(slice_info, slice_indices, rs, alpha=0.4):
    """
    Plots multiple slices with density fields and point positions for different environment types.

    Parameters:
    - slice_info (dict): Dictionary containing positions and density fields for each slice and environment type.
    - slice_indices (list): List of slice indices to be plotted.
    - rs (float): Value of Rs (scale parameter).
    - alpha (float): Alpha value for point transparency (default is 0.4).

    Returns:
    - None

    Notes:
    - The slice_info parameter is a dictionary with keys representing slice indices and environment types.
    - The slice_indices parameter is a list of slice indices to be plotted.
    - The rs parameter is the value of Rs (scale parameter) to be displayed in the plot.
    - Density fields and point positions are overlaid for each environment type on each slice.
    - The plot includes a title, labels, and a legend.

    Example:
    >>> slice_info = {'1': [array1_void, array1_sheet, array1_filament, array1_cluster],
    ...               '2': [array2_void, array2_sheet, array2_filament, array2_cluster],
    ...               'density': [density_array1, density_array2]}
    >>> slice_indices = [1, 2]
    >>> rs = 5.0
    >>> plot_slices(slice_info, slice_indices, rs)
    """
    labels = ['Void', 'Sheet', 'Filament', 'Cluster']

    num_plots = len(slice_indices)
    
    if num_plots <= 3:
        num_rows = 1
        num_cols = num_plots
    
    elif num_plots % 3 == 0:
        num_rows = num_plots // 3
        num_cols = 3
    
    else:
        num_rows = (num_plots // 3) + 1
        num_cols = 3

    fig = plt.figure(figsize=(20, 14), dpi=400)
    gs = gridspec.GridSpec(num_rows, num_cols, width_ratios=[0.5, 0.5, 0.5], height_ratios=[1, 1])
    fig.subplots_adjust(wspace=0.2)

    for idx, slice_index in enumerate(slice_indices):
        row = idx // num_cols
        col = idx % num_cols

        ax = plt.subplot(gs[row, col])

        for i in range(len(labels)):
            ax.imshow(slice_info['density'][idx], cmap='Greys', extent=[0, 100, 0, 100], origin='lower')
            ax.scatter(slice_info[str(slice_index)][i][:, 0], slice_info[str(slice_index)][i][:, 1],\
                       s=0.02, alpha=alpha, label=labels[i])
            ax.tick_params(axis='both', which='both', left=True, right=True, top=True, bottom=True, direction='in', labelsize=15)
            ax.set_xlabel(r'$h^{-1}~Mpc$', fontsize=15)
            ax.set_ylabel(r'$h^{-1}~Mpc$', fontsize=15)
        ax.set_title(r'${:.2f}~h^{{-1}}~Mpc$'.format(slice_indices[idx] * 100 / 512), fontsize=15)

    plt.suptitle(r'$R_s = {}~h^{{-1}}~Mpc$'.format(rs), fontsize=25)

    # Place the legend outside the plots
    plt.legend(bbox_to_anchor=(1.05, 0.5), fontsize=18, markerscale=75.0, ncol=1)

    plt.show()


In [ ]:
slice_list = [236, 246, 256, 266, 276]
all_info_054 = get_env_pos_and_density(class_054, slice_list, rho054)

In [ ]:
plot_slices(all_info_054, slice_list, 0.54, 0.5)

#### $R_s = 0.76 h^{-1} Mpc$ for 5 consecutive slices

In [ ]:
all_info_076 = get_env_pos_and_density(class_076, slice_list, rho076)

In [ ]:
plot_slices(all_info_076, slice_list, 0.76)

#### $R_s = 1.16 h^{-1} Mpc$ for 5 consecutive slices

In [ ]:
all_info_116 = get_env_pos_and_density(class_116, slice_list, rho116)

In [ ]:
plot_slices(all_info_076, slice_list, 1.167)

#### $R_s = 1.30 h^{-1} Mpc$ for 5 consecutive slices

In [ ]:
all_info_13 = get_env_pos_and_density(class_13, slice_list, rho13)

In [ ]:
plot_slices(all_info_13, slice_list, 1.3)

In [ ]:
def plot_2d_env(slice_info, env_index, slice_indices, rs):
    """
    Plots 2D positions of a specific environment type across multiple slices.

    Parameters:
    - slice_info (dict): Dictionary containing positions and density fields for each slice and environment type.
    - env_index (int): Index representing the environment type ('Void', 'Sheet', 'Filament', 'Cluster').
    - slice_indices (list): List of slice indices to be plotted.
    - rs (float): Value of Rs (scale parameter).

    Returns:
    - None

    Notes:
    - The slice_info parameter is a dictionary with keys representing slice indices and environment types.
    - The env_index parameter is an index representing the environment type to be plotted.
    - The slice_indices parameter is a list of slice indices to be plotted.
    - The rs parameter is the value of Rs (scale parameter) to be displayed in the plot.
    - Positions are plotted in 2D for the specified environment type across multiple slices.
    - The plot includes labels, a title, and different colors for each slice.

    Example:
    >>> slice_info = {'1': [array1_void, array1_sheet, array1_filament, array1_cluster],
    ...               '2': [array2_void, array2_sheet, array2_filament, array2_cluster]}
    >>> env_index = 2
    >>> slice_indices = [1, 2]
    >>> rs = 5.0
    >>> plot_2d_env(slice_info, env_index, slice_indices, rs)
    """
    labels = ['Void', 'Sheet', 'Filament', 'Cluster']
    
    plt.figure(figsize=(10, 7), dpi=200)

    # Create a unique color for each combination of label and slice
    colors = itertools.cycle(plt.cm.tab10.colors)
    
    for idx, slice_index in enumerate(slice_indices):
        color = next(colors)
        
        plt.scatter(slice_info[str(slice_index)][env_index][:, 1], slice_info[str(slice_index)][env_index][:, 0],\
                    s=0.08, alpha=0.8, label=f'Slice {idx+1}', color=color)
        

    plt.xlabel(r'$h^{-1}~Mpc$', fontsize=12)
    plt.ylabel(r'$h^{-1}~Mpc$', fontsize=12)
    plt.title(r'{} Environment: $R_s = {}~h^{{-1}}~Mpc$'.format(labels[env_index], rs), fontsize=15)
    
    plt.show()


In [ ]:
def plot_3d_env(slice_info, env_index, slice_indices, rs, side):
    """
    Plots 3D positions of a specific environment type across multiple slices.

    Parameters:
    - slice_info (dict): Dictionary containing positions and density fields for each slice and environment type.
    - env_index (int): Index representing the environment type ('Void', 'Sheet', 'Filament', 'Cluster').
    - slice_indices (list): List of slice indices to be plotted.
    - rs (float): Value of Rs (scale parameter).
    - side (str): Projection side ('Front' or 'Side').

    Returns:
    - None

    Notes:
    - The slice_info parameter is a dictionary with keys representing slice indices and environment types.
    - The env_index parameter is an index representing the environment type to be plotted.
    - The slice_indices parameter is a list of slice indices to be plotted.
    - The rs parameter is the value of Rs (scale parameter) to be displayed in the plot.
    - The side parameter specifies the projection side ('Front' or 'Side').
    - Positions are plotted in 3D for the specified environment type across multiple slices.
    - The plot includes labels, a title, and different colors for each slice.

    Example:
    >>> slice_info = {'1': [array1_void, array1_sheet, array1_filament, array1_cluster],
    ...               '2': [array2_void, array2_sheet, array2_filament, array2_cluster]}
    >>> env_index = 2
    >>> slice_indices = [1, 2]
    >>> rs = 5.0
    >>> side = 'Front'
    >>> plot_3d_env(slice_info, env_index, slice_indices, rs, side)
    """
    labels = ['Void', 'Sheet', 'Filament', 'Cluster']
    
    fig = plt.figure(figsize=(10, 7), dpi=200)
    ax_3d = fig.add_subplot(111, projection='3d')

    # Create a unique color for each combination of label and slice
    colors = itertools.cycle(plt.cm.tab10.colors)
    
    a = np.linspace(0.2, 0.7, len(slice_indices))
    
    for idx, slice_index in enumerate(slice_indices):
        z_coord = slice_index * 100 / 512
        color = next(colors)
        
        if str(side).lower()[0] == 'f':
            ax_3d.scatter(slice_info[str(slice_index)][env_index][:, 0], slice_info[str(slice_index)][env_index][:, 1], z_coord,\
                          s=0.08, alpha=0.8, label=f'Slice {idx+1}', color=color)
        
        else:
            ax_3d.scatter(slice_info[str(slice_index)][env_index][:, 1], slice_info[str(slice_index)][env_index][:, 0], z_coord,\
                          s=0.08, alpha=0.8, label=f'Slice {idx+1}', color=color)
        

    ax_3d.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
    ax_3d.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
    ax_3d.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
    ax_3d.set_title(r'{} Environment: $R_s = {}~h^{{-1}}~Mpc$ ({})'.format(labels[env_index], rs, side), fontsize=15)
    
    ax_3d.view_init(elev=20)

    plt.show()

In [ ]:
from mayavi import mlab

# Function to create volume rendering for a specific environment
def plot_environment(environment_code):
    # Create a binary mask for the specific environment
    environment_mask = (class_054 == environment_code)

    # Create volume rendering
    src = mlab.pipeline.scalar_field(environment_mask)
    vol = mlab.pipeline.volume(src)

    # Customize the visualization settings if needed
    # For example, adjust colormap, opacity, etc.

    # Show the plot
    mlab.show()

# Generate volume renderings for each environment
plot_environment(0)  # Voids
# plot_environment(1)  # Sheets
# plot_environment(2)  # Filaments
# plot_environment(3)  # Clusters

For $R_s = 0.54 h^{-1}~Mpc$

In [ ]:
env_slices = np.arange(246, 277, 1)
info_054 = get_env_pos_and_density(class_054, env_slices, rho054)

In [ ]:
plot_3d_env(info_054, 0, env_slices, 0.54, 'front')

In [ ]:
plot_3d_env(info_054, 0, env_slices, 0.54, 'back')

In [ ]:
plot_2d_env(info_054, 0, env_slices, 0.54)

In [ ]:
plot_3d_env(info_054, 1, env_slices, 0.54, 'front')

In [ ]:
plot_3d_env(info_054, 1, env_slices, 0.54, 'back')

In [ ]:
plot_2d_env(info_054, 1, env_slices, 0.54)

In [ ]:
plot_3d_env(info_054, 2, env_slices, 0.54, 'front')

In [ ]:
plot_3d_env(info_054, 2, env_slices, 0.54, 'back')

In [ ]:
plot_2d_env(info_054, 2, env_slices, 0.54)

In [ ]:
plot_3d_env(info_054, 3, env_slices, 0.54, 'front')

In [ ]:
plot_3d_env(info_054, 3, env_slices, 0.54, 'back')

In [ ]:
plot_2d_env(info_054, 3, env_slices, 0.54)

For $R_s = 1.3 h^{-1}~Mpc$

In [ ]:
info_13 = get_env_pos_and_density(class_13, env_slices, rho13)

In [ ]:
plot_3d_env(info_13, 0, env_slices, 1.3, 'front')

In [ ]:
plot_3d_env(info_13, 0, env_slices, 1.3, 'back')

In [ ]:
plot_2d_env(info_13, 0, env_slices, 0.54)

In [ ]:
plot_3d_env(info_13, 1, env_slices, 1.3, 'front')

In [ ]:
plot_3d_env(info_13, 1, env_slices, 1.3, 'back')

In [ ]:
plot_2d_env(info_13, 1, env_slices, 0.54)

In [ ]:
plot_3d_env(info_13, 2, env_slices, 1.3, 'front')

In [ ]:
plot_3d_env(info_13, 2, env_slices, 1.3, 'back')

In [ ]:
plot_2d_env(info_13, 2, env_slices, 1.3)

In [ ]:
plot_3d_env(info_13, 3, env_slices, 1.3, 'front')

In [ ]:
plot_3d_env(info_13, 3, env_slices, 1.3, 'back')

In [ ]:
plot_2d_env(info_13, 3, env_slices, 0.54)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import itertools

def plot_3d_env(slice_info, env_values, slice_indices, rs_values):

    fig, axs = plt.subplots(2, 2, figsize=(12, 10), dpi=200, subplot_kw={'projection': '3d'})
    axs = axs.flatten()

    for ax, env, rs in zip(axs, env_values, rs_values):

#         colors = itertools.cycle(plt.cm.tab20.colors)

        for idx, slice_index in enumerate(slice_indices):
            z_coord = slice_index * 100 / 512
            color = next(colors)
            ax.scatter(slice_info[str(slice_index)][env][:, 0], slice_info[str(slice_index)][env][:, 1], z_coord,
                       s=0.08, alpha=0.8, label=f'Slice {slice_index}')

        ax.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
        ax.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
        ax.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
        ax.set_title(fr'$R_s = {rs}~h^{{-1}}~Mpc$ (X-Y)', fontsize=16)

    # Adjust layout
    plt.tight_layout()
    plt.show()

# Example usage
env_values = [0, 1, 2, 3]  # Update with your specific env_values
slice_indices = [0, 1, 2, 3]  # Update with your specific slice_indices
rs_values = [1.0, 2.0, 3.0, 4.0]  # Update with your specific rs_values

# Assuming slice_info is a dictionary containing relevant information
# slice_info = ...

plot_3d_env(slice_info, env_values, slice_indices, rs_values)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import itertools

labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_all_13 = [env_pos_13_4, env_pos_13_5, env_pos_13_6, env_pos_13_7, env_pos_13_8]
d_scales_13 = [d_field_13_4, d_field_13_5, d_field_13_6, d_field_13_7, d_field_13_8]
slice_num = np.array([236, 246, 256, 266, 276]) * 100 / 512

num_slices = len(env_pos_all_13)

fig = plt.figure(figsize=(10, 7), dpi=400)
ax_3d = fig.add_subplot(111, projection='3d')

# Create a unique color for each combination of label and slice
colors = itertools.cycle(plt.cm.tab20.colors)

for idx, ev_all_pos in enumerate(env_pos_all_13):
    for i, label in enumerate(labels):
        z_coord = slice_num[idx] 
        color = next(colors)
        ax_3d.scatter(ev_all_pos[i][:, 0], ev_all_pos[i][:, 1], z_coord, s=0.08, alpha=0.8, label=f'{label} - Slice {idx+1}', color=color)

ax_3d.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_title(r'$R_s = 1.3~h^{-1}~Mpc$ (X-Y)', fontsize=20)

ax_3d.legend(bbox_to_anchor=(1.15, 0.8), fontsize=8, markerscale=70, ncol=1)

# ax_3d.view_init(elev=20)

plt.show()


In [ ]:
labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_all_13 = [env_pos_13_4, env_pos_13_5, env_pos_13_6, env_pos_13_7, env_pos_13_8]

d_scales_13 = [d_field_13_4, d_field_13_5, d_field_13_6, d_field_13_7, d_field_13_8]

slice_num = np.array([236, 246, 256, 266, 276]) * 100 / 512

num_slices = len(env_pos_all_13)

fig = plt.figure(figsize=(10, 7), dpi=400)
ax_3d = fig.add_subplot(111, projection='3d')

for idx, ev_all_pos in enumerate(env_pos_all_13):
    for i, label in enumerate(labels):
        z_coord = slice_num[idx] 
        ax_3d.scatter(ev_all_pos[i][:, 0], ev_all_pos[i][:, 1], z_coord, s=0.02, alpha=0.8, label=f'{label} - Slice {idx+1}')

ax_3d.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_title(r'$R_s = 1.3~h^{-1}~Mpc$ (X-Y)', fontsize=20)

ax_3d.legend(bbox_to_anchor=(1.15, 0.8), fontsize=8, markerscale=70, ncol=1)

# ax_3d.view_init(elev=20)

plt.show()


In [ ]:
labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_all_13 = [env_pos_13_4, env_pos_13_5, env_pos_13_6, env_pos_13_7, env_pos_13_8]
d_scales_13 = [d_field_13_4, d_field_13_5, d_field_13_6, d_field_13_7, d_field_13_8]
slice_num = np.array([236, 246, 256, 266, 276]) * 100 / 512

num_slices = len(env_pos_all_13)

fig = plt.figure(figsize=(10, 7), dpi=400)
ax_3d = fig.add_subplot(111, projection='3d')

for idx, ev_all_pos in enumerate(env_pos_all_13):
    for i, label in enumerate(labels):
        z_coord = slice_num[idx] 
        ax_3d.scatter(ev_all_pos[i][:, 1], ev_all_pos[i][:, 0], z_coord, s=0.02, alpha=0.4, label=f'{label} - Slice {idx+1}')

ax_3d.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_title(r'$R_s = 1.3~h^{-1}~Mpc$ (Y-X)', fontsize=20)

# Place the legend outside the plot
ax_3d.legend(bbox_to_anchor=(1.15, 0.8), fontsize=8, markerscale=70, ncol=1)

# Set the viewing angle (elevation, azimuth)
# ax_3d.view_init(elev=20)

plt.show()


In [ ]:
labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_all_13 = [env_pos_13_4, env_pos_13_5, env_pos_13_6, env_pos_13_7, env_pos_13_8]
d_scales_13 = [d_field_13_4, d_field_13_5, d_field_13_6, d_field_13_7, d_field_13_8]
slice_num = np.array([236, 246, 256, 266, 276]) * 100 / 512
color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

num_slices = len(env_pos_all_13)

fig = plt.figure(figsize=(10, 7), dpi=400)
ax_3d = fig.add_subplot(111, projection='3d')

for idx, ev_all_pos in enumerate(env_pos_all_13):
    for i, label in enumerate(labels):
        z_coord = slice_num[idx]  # Adjust the z-coordinate for each slice
        ax_3d.scatter(ev_all_pos[i][:, 0], ev_all_pos[i][:, 1], z_coord, s=0.02, alpha=0.3, color = color[i], label=f'{label} - Slice {idx+1}')

ax_3d.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_title(r'$R_s = 1.3~h^{-1}~Mpc$ (X-Y)', fontsize=20)

# Place the legend outside the plot
plt.legend(bbox_to_anchor=(1.15, 0.8), fontsize=8, markerscale=70, ncol=1)

# Set the viewing angle (elevation, azimuth)
# ax_3d.view_init(elev=20)

plt.show()


In [ ]:
labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_all_13 = [env_pos_13_4, env_pos_13_5, env_pos_13_6, env_pos_13_7, env_pos_13_8]
d_scales_13 = [d_field_13_4, d_field_13_5, d_field_13_6, d_field_13_7, d_field_13_8]
slice_num = np.array([236, 246, 256, 266, 276]) * 100 / 512
color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

num_slices = len(env_pos_all_13)

fig = plt.figure(figsize=(10, 7), dpi=400)
ax_3d = fig.add_subplot(111, projection='3d')

for idx, ev_all_pos in enumerate(env_pos_all_13):
    for i, label in enumerate(labels):
        z_coord = slice_num[idx]  # Adjust the z-coordinate for each slice
        ax_3d.scatter(ev_all_pos[i][:, 1], ev_all_pos[i][:, 0], z_coord, s=0.02, alpha=0.3, color = color[i], label=f'{label} - Slice {idx+1}')

ax_3d.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_title(r'$R_s = 1.3~h^{-1}~Mpc$ (Y-X)', fontsize=20)

# Place the legend outside the plot
plt.legend(bbox_to_anchor=(1.15, 0.8), fontsize=8, markerscale=70, ncol=1)

# Set the viewing angle (elevation, azimuth)
# ax_3d.view_init(elev=20)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec

labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_all_13 = [env_pos_13_4, env_pos_13_5, env_pos_13_6, env_pos_13_7, env_pos_13_8]
d_scales_13 = [d_field_13_4, d_field_13_5, d_field_13_6, d_field_13_7, d_field_13_8]
slice_num = np.array([236, 246, 256, 266, 276]) * 100 / 512

num_slices = len(env_pos_all_13)

fig = plt.figure(figsize=(10, 7), dpi=400)
ax_3d = fig.add_subplot(111, projection='3d')

# Plot density field surface
x_density, y_density = np.meshgrid(np.linspace(0, 100, d_scales_13[0].shape[0]), np.linspace(0, 100, d_scales_13[0].shape[1]))
for idx, d_scale in enumerate(d_scales_13):
    z_coord = slice_num[idx]  # Adjust the z-coordinate for each slice
    ax_3d.plot_surface(x_density, y_density, z_coord * np.ones_like(d_scale), facecolors=plt.cm.viridis(d_scale),
                       alpha=0.7, rstride=5, cstride=5)

# # Plot scatter points
# for idx, ev_all_pos in enumerate(env_pos_all_13):
#     for i, label in enumerate(labels):
#         z_coord = slice_num[idx]  # Adjust the z-coordinate for each slice
#         ax_3d.scatter(ev_all_pos[i][:, 0], ev_all_pos[i][:, 1], z_coord, s=0.02, alpha=0.4, label=f'{label} - Slice {idx+1}')

ax_3d.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_title(r'$R_s = 1.3~h^{-1}~Mpc$ (X-Y)', fontsize=20)

# Place the legend outside the plot
# ax_3d.legend(bbox_to_anchor=(1.05, 0.5), fontsize=8, markerscale=7.0, ncol=1)

# Set the viewing angle (elevation, azimuth)
# ax_3d.view_init(elev=20)

plt.show()


In [ ]:
labels = ['Void', 'Sheet', 'Filament', 'Cluster']
env_pos_all_13 = [env_pos_13_4, env_pos_13_5, env_pos_13_6, env_pos_13_7, env_pos_13_8]
d_scales_13 = [d_field_13_4, d_field_13_5, d_field_13_6, d_field_13_7, d_field_13_8]
slice_num = np.array([236, 246, 256, 266, 276]) * 100 / 512

num_slices = len(env_pos_all_13)

fig = plt.figure(figsize=(10, 7), dpi=400)
ax_3d = fig.add_subplot(111, projection='3d')

# Plot density field surface
x_density, y_density = np.meshgrid(np.linspace(0, 100, d_scales_13[0].shape[0]), np.linspace(0, 100, d_scales_13[0].shape[1]))
for idx, d_scale in enumerate(d_scales_13):
    z_coord = slice_num[idx]  # Adjust the z-coordinate for each slice
    ax_3d.plot_surface(y_density, x_density, z_coord * np.ones_like(d_scale), facecolors=plt.cm.viridis(d_scale), 
                       alpha=0.7, rstride=5, cstride=5)

ax_3d.set_xlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_ylabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_zlabel(r'$h^{-1}~Mpc$', fontsize=12)
ax_3d.set_title(r'$R_s = 1.3~h^{-1}~Mpc$ (Y-X)', fontsize=15)

# Place the legend outside the plot
# ax_3d.legend(bbox_to_anchor=(1.05, 0.5), fontsize=8, markerscale=7.0, ncol=1)

# Set the viewing angle (elevation, azimuth)
# ax_3d.view_init(elev=20)

plt.show()


### Extra/miscelleneous plots

In [ ]:
fig, ax = plt.subplots(6, 2, figsize = (10, 15), dpi = 150)

ax = ax.flatten()

for i in range(len(classifications)):
    ax[i].hist(classifications[i].flatten(), bins=[-0.5, 0.5, 1.5, 2.5, 3.5], density = True, edgecolor='black', align='mid', color='lightblue')
    ax[i].set_title(f'Smoothing scale $R_s = {sm_scales[i]}$'+'$~h^{-1} Mpc$', fontsize = 13)
    ax[i].set_xticks(range(4), labels)
    ax[i].grid(alpha = 0.5)

fig.tight_layout()
plt.show()

In [ ]:
import plotly.graph_objects as go

# Sample 3D data
x_values = [1, 2, 3, 4, 5]
y_values = [10, 15, 13, 17, 21]
z_values = [5, 8, 10, 12, 15]

# Create a trace (scatter plot in 3D)
trace = go.Scatter3d(x=x_values, y=y_values, z=z_values, mode='markers', marker=dict(size=10, color='blue'))

# Create layout
layout = go.Layout(title='Simple 3D Scatter Plot', scene=dict(xaxis=dict(title='X-axis'), yaxis=dict(title='Y-axis'), zaxis=dict(title='Z-axis')))

# Create figure
fig = go.Figure(data=[trace], layout=layout)

# Show the plot
fig.show()